<a href="https://colab.research.google.com/github/jack-cao-623/python_learning/blob/main/Python_Random_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Random Forests

## Tutorial

In [22]:
# get kaggle data

# first, mount google drive

# copy .json file
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

# get data
! kaggle datasets download gunjanpathak/melb-data -f melb_data.csv

# unzip
! unzip melb_data.csv.zip

melb_data.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  melb_data.csv.zip
replace melb_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [32]:
# libraries needed
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [33]:
# load data
melb_data = pd.read_csv('melb_data.csv')

# remove rows (axis = 0) with missing data
melb_data = melb_data.dropna(axis = 0)

# view
melb_data.head()

,Unnamed: 0,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
4,6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
6,11,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,14,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0


In [34]:
# define features, X, and prediction target, y
X = melb_data[['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']]
y = melb_data[['Price']]

In [35]:
# view features
X

,Rooms,Bathroom,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
1,2,1.0,156.0,79.00,1900.0,-37.80790,144.99340
2,3,2.0,134.0,150.00,1900.0,-37.80930,144.99440
4,4,1.0,120.0,142.00,2014.0,-37.80720,144.99410
6,3,2.0,245.0,210.00,1910.0,-37.80240,144.99930
7,2,1.0,256.0,107.00,1890.0,-37.80600,144.99540
...,...,...,...,...,...,...,...
15388,3,2.0,972.0,149.00,1996.0,-37.51232,145.13282
15389,3,1.0,179.0,115.00,1890.0,-37.86558,144.90474
15390,1,1.0,0.0,35.64,1967.0,-37.85588,144.89936
15392,2,1.0,0.0,61.60,2012.0,-37.85581,144.99025


In [36]:
# descriptive statistics on features
X.describe()
  # all numeric

,Rooms,Bathroom,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,141.568645,1964.081988,-37.807904,144.990201
std,0.971079,0.711362,897.449881,90.834824,38.105673,0.075850,0.099165
min,1.000000,1.000000,0.000000,0.000000,1196.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,91.000000,1940.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,124.000000,1970.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,170.000000,2000.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,3112.000000,2018.000000,-37.457090,145.526350


In [37]:
# view prediction target, y
y

,Price
1,1035000.0
2,1465000.0
4,1600000.0
6,1876000.0
7,1636000.0
...,...
15388,601000.0
15389,1050000.0
15390,385000.0
15392,560000.0


In [38]:
# descriptive statistics on y
y.describe()

,Price
count,6.196000e+03
mean,1.068828e+06
std,6.751564e+05
min,1.310000e+05
25%,6.200000e+05
50%,8.800000e+05
75%,1.325000e+06
max,9.000000e+06


In [39]:
# split into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [40]:
# specify model
model_rf = RandomForestRegressor(random_state = 1)

# fit model
model_rf.fit(X_train, y_train)

# predict values
y_predicted = model_rf.predict(X_test)

# mean absolute error
mae = mean_absolute_error(y_predicted, y_test)
print(mae)

<ipython-input-40-0a14ceb1f1f6>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf.fit(X_train, y_train)


191669.7536453626


## Exercise

In [41]:
# get training data
! kaggle competitions download home-data-for-ml-course -f train.csv

  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 4.86MB/s]


In [42]:
# load data
home_data = pd.read_csv('train.csv')

# view
home_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [43]:
# define prediciton target, y, and features, X
y = home_data[['SalePrice']]
X = home_data[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']]

In [44]:
# split X and y into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [47]:
# specify random forst
rf_home_data = RandomForestRegressor(random_state = 1)

# fit model
rf_home_data.fit(X_train, y_train)

# predicted values
y_predicted = rf_home_data.predict(X_test)

# MAE
print(mean_absolute_error(y_predicted, y_test))

<ipython-input-47-6c23bec25a40>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_home_data.fit(X_train, y_train)


21857.15912981083
